### **LangChain Runnable 개요**

1. Runnable이란?

- Runnable은 LangChain에서 데이터 흐름을 조합하고 실행할 수 있도록 하는 핵심 인터페이스
- AI 파이프라인을 구성할 때 다양한 연산을 조합하여 체인을 만들 수 있음

In [18]:
# 기본 LCEL 체인
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq


prompt = PromptTemplate.from_template("{input}에 대해서 알려줘")

llm = ChatGroq(model = 'gemma2-9b-it',
               max_tokens=256)

chain = prompt | llm

chain.invoke('프로그래밍 언어')   # {'input': '프로그래밍 언어'}  => 변수가 1개일땐 딕셔너리를 사용하지 않아도 됨


AIMessage(content='## 프로그래밍 언어: 컴퓨터와 대화하는 언어\n\n프로그래밍 언어는 컴퓨터가 이해하고 실행할 수 있는 명령어를 사용하여 컴퓨터 프로그램을 작성하는 데 사용하는 **formal language**입니다. 마치 사람들이 서로 소통하는 언어처럼, 프로그래밍 언어를 통해 사람들은 컴퓨터에 특정 작업을 수행하도록 지시할 수 있습니다.\n\n**프로그래밍 언어의 종류:**\n\n프로그래밍 언어는 다양한 목적으로 개발되었으며, 그 종류는 매우 다양합니다. 몇 가지 주요 유형은 다음과 같습니다:\n\n* **일반적 목적 언어 (General-Purpose Language):** Python, Java, C++, JavaScript 등 다양한 작업에 사용 가능한 언어입니다. 웹 개발, 데이터 분석, 게임 개발 등 다양한 분야에서 사용됩니다.\n* **구체적 목적 언어 (Special-Purpose Language):** 특정 작업에 특화된 언어입니다. 예를 들어, SQL은 데이터베이스를 관리하는 데 사용되고, R은 통', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 256, 'prompt_tokens': 21, 'total_tokens': 277, 'completion_time': 0.465454545, 'prompt_time': 0.002134766, 'queue_time': 0.018743863, 'total_time': 0.467589311}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'length', 'logprobs': None}, id='run-0eb1382b-0fd8-44e3-a234-7632c5b4cd1d-0', usage_metadata={'input_tokens': 21, 'output_tokens': 256, 'tot

### **RunnablePassthrough**

- 입력을 그대로 반환하는 Runnable

- 체인 내에서 원본 데이터를 유지할 때 유용

- 기본적으로 Dict 형태로 전달하지만 RunnablePassthrough를 사용하면 Dict 형태로 전달하지 않아도 됨

- 데이터를 받아서 전달 해준다 

`RunnablePassthrough` 는 `runnable` 객체이며, `runnable` 객체는 `invoke()` 메소드를 사용하여 별도 실행이 가능

In [19]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({'input': '프로그래밍 언어'})

{'input': '프로그래밍 언어'}

In [ ]:
# 체인구성 
runnable_chain = {"input": RunnablePassthrough()} | prompt | llm

runnable_chain.invoke('프로그래밍 언어')

AIMessage(content='## 프로그래밍 언어에 대해 알려드릴게요! \n\n프로그래밍 언어는 컴퓨터에게 명령어를 전달하는 데 사용되는 특수한 언어입니다. \n\n**쉽게 말해, 우리가 컴퓨터가 하고 싶은 일을 설명하는 언어죠.**\n\n💻 **프로그래밍 언어의 종류**:\n\n* **구문 언어 (Syntax-based languages)**:  영어와 비슷한 문법 규칙을 따릅니다. (예: Python, Java, C++)\n* **순서 언어 (Order-based languages)**: 명령어를 순서대로 실행합니다. (예: BASIC)\n* **비순서 언어 (Non-order-based languages)**: 명령어의 순서는 중요하지 않습니다. (예: Prolog)\n\n* **적용 언어 (Application languages)**: 특정 용도로 만들어진 언어입니다. (예: 웹 개발에 사용되는 HTML, CSS, JavaScript)\n* **시스템 언어 (System languages)**: 컴퓨터 하드웨어를 제어하는 데 사용됩니다. (예: Assembly', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 256, 'prompt_tokens': 21, 'total_tokens': 277, 'completion_time': 0.465454545, 'prompt_time': 0.002242305, 'queue_time': 0.017731114, 'total_time': 0.46769685}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'length', 'logprobs': None}, id='run-bc84e646-fca1-467a-a30e-9fe134fe7ad2-0', usage_metadata={'input_tokens': 21, 'output

`RunnablePassthrough.assign()`

- 입력 값으로 들어온 값의 key/value 쌍과 새롭게 할당된 key/value 쌍을 합칩.

In [22]:
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 1, 'new_num': 3}

### **RunnableParallel**

- RunnableParallel은 LangChain에서 여러 개의 Runnable을 동시에 실행할 수 있도록 도와주는 컴포넌트 
- 입력 데이터를 여러 개의 Runnable에 병렬적으로 전달하고, 그 결과를 딕셔너리 형태로 반환합
- 체인도 엮을 수 있음

In [31]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda

# RunnableParallel을 사용하여 여러 개의 변환 작업을 병렬 실행
runnable = RunnableParallel(

    # 원본 입력 그대로 전달
    original=RunnablePassthrough(),
    
    # 대문자로 변환
    uppercase=RunnableLambda(lambda x: x["text"].upper()),

    # 'extra' 필드를 추가하여 텍스트 길이 및 역순 변환된 텍스트 저장
    extra=RunnablePassthrough.assign(
        length=lambda x: len(x["text"]),    # 텍스트 길이 추가
        reversed=lambda x: x["text"][::-1]  # 텍스트 역순 변환
    ),
)

# 실행
result = runnable.invoke({"text": "LangChain is powerful!"})

result

{'original': {'text': 'LangChain is powerful!'},
 'uppercase': 'LANGCHAIN IS POWERFUL!',
 'extra': {'text': 'LangChain is powerful!',
  'length': 22,
  'reversed': '!lufrewop si niahCgnaL'}}

In [38]:
# Chain 구성 

chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 수도는?")
    | llm
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country} 의 면적은?")
    | llm
)

combined_chain = RunnableParallel(capital=chain1, area=chain2)
result = combined_chain.invoke("대한민국")

In [59]:
result

{'capital': AIMessage(content='대한민국의 수도는 **서울**입니다. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 17, 'total_tokens': 32, 'completion_time': 0.027272727, 'prompt_time': 0.001915277, 'queue_time': 0.019418392, 'total_time': 0.029188004}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7093ca2c-7817-47d9-84a2-88f2b5e2f125-0', usage_metadata={'input_tokens': 17, 'output_tokens': 15, 'total_tokens': 32}),
 'area': AIMessage(content='대한민국의 면적은 약 **100,210 제곱 킬로미터** 입니다. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 18, 'total_tokens': 48, 'completion_time': 0.054545455, 'prompt_time': 0.001904128, 'queue_time': 0.017587140999999997, 'total_time': 0.056449583}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-585d5ea0-9769-417e-bf69-14

In [54]:
for key, value in result.items():
    print(value.content)
    

대한민국의 수도는 **서울**입니다. 

대한민국의 면적은 약 **100,210 제곱 킬로미터** 입니다. 



### **RunnableLambda란**

- RunnableLambda는 간단한 함수를 Runnable로 변환하는 역할을 하는 LangChain의 핵심 클래스
- Lambda 함수와 사용자 정의 함수를 Runnable 체인에 포함시켜 실행

In [134]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime


def get_today(a):
    print(f'입력받은 변수의 값 : {a}')
    # 오늘 날짜를 가져오기
    return datetime.today().strftime("%b-%d")


# 오늘 날짜를 출력
get_today(0)

입력받은 변수의 값 : 0


'Mar-09'

In [143]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from operator import itemgetter

# prompt 와 llm 을 생성합니다.
prompt = PromptTemplate.from_template(
    "{today} 가 생일인 유명인 {n} 명을 나열하세요. 생년월일을 표기해 주세요."
)

# chain 을 생성합니다.
chain = (
    # {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}   # 이렇게 넣으면 프롬프트에 {'n' : 3} 이렇게 들어갔음
    {"today": RunnableLambda(get_today), "n": itemgetter("n")}   # 이렇게 넣으면 프롬프트에 3 이렇게 들어감  특정키를 추출하는 객체
    | prompt
    | llm
    | StrOutputParser()
)

# 출력
print(chain.invoke({"n": 3}))

입력받은 변수의 값 : {'n': 3}
  마치 3월 9일 생일을 가진 유명 인사 3명은 다음과 같습니다.

* **Justin Bieber** (1994년 3월 1일)
* **Emma Watson** (1990년 4월 15일)
* **Ryan Reynolds** (1976년 10월 23일) 



